In [39]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import random
from sklearn import linear_model, metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split as split

In [49]:
data= pd.read_csv("train_V2.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 983.9+ MB


In [5]:
cor= data.corr()
cor.style.background_gradient().set_precision(2)

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
assists,1,0.31,0.41,0.3,0.2,0.23,-0.29,0.039,0.32,0.24,0.26,-0.019,-0.15,-0.15,-0.016,0.2,0.11,0.012,0.023,0.0061,0.058,0.29,0.24,0.024,0.3
boosts,0.31,1,0.52,0.36,0.33,0.54,-0.55,0.0083,0.5,0.41,0.42,0.072,-0.014,-0.013,0.023,0.25,0.33,0.035,0.11,0.013,0.087,0.64,0.41,-0.0076,0.63
damageDealt,0.41,0.52,1,0.74,0.61,0.34,-0.68,0.05,0.89,0.7,0.56,-0.0068,-0.041,-0.04,-0.0015,0.26,0.14,0.052,0.037,0.015,0.081,0.4,0.35,0.018,0.44
DBNOs,0.3,0.36,0.74,1,0.47,0.27,-0.56,0.043,0.71,0.65,0.45,-0.014,-0.27,-0.27,-0.0035,0.3,0.1,0.036,0.017,0.069,0.06,0.28,0.22,0.011,0.28
headshotKills,0.2,0.33,0.61,0.47,1,0.2,-0.47,0.024,0.67,0.51,0.45,-0.018,0.0092,0.0094,0.0051,0.15,0.076,0.013,0.029,0.0087,0.039,0.25,0.22,0.005,0.28
heals,0.23,0.54,0.34,0.27,0.2,1,-0.39,-0.0026,0.31,0.27,0.26,0.11,-0.065,-0.064,0.019,0.24,0.3,0.025,0.08,0.035,0.063,0.43,0.31,-0.011,0.43
killPlace,-0.29,-0.55,-0.68,-0.56,-0.47,-0.39,1,-0.023,-0.73,-0.81,-0.54,-0.0058,0.017,0.019,-0.015,-0.27,-0.24,-0.057,-0.089,-0.038,-0.074,-0.59,-0.49,-0.0024,-0.72
killPoints,0.039,0.0083,0.05,0.043,0.024,-0.0026,-0.023,1,0.036,0.037,0.014,-0.1,-0.045,-0.042,-0.98,0.012,-0.029,-0.0042,-0.0046,-0.04,-0.0097,0.0046,-0.004,0.98,0.013
kills,0.32,0.5,0.89,0.71,0.67,0.31,-0.73,0.036,1,0.8,0.6,-0.031,-0.0048,-0.0041,0.0082,0.25,0.11,0.064,0.039,0.018,0.08,0.37,0.34,0.0069,0.42
killStreaks,0.24,0.41,0.7,0.65,0.51,0.27,-0.81,0.037,0.8,1,0.52,-0.035,-0.037,-0.036,0.0034,0.24,0.098,0.066,0.035,0.028,0.064,0.33,0.28,0.0086,0.38


In [50]:
# One hot encode matchType
data = pd.get_dummies(data, columns=['matchType'])


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 44 columns):
Id                            object
groupId                       object
matchId                       object
assists                       int64
boosts                        int64
damageDealt                   float64
DBNOs                         int64
headshotKills                 int64
heals                         int64
killPlace                     int64
killPoints                    int64
kills                         int64
killStreaks                   int64
longestKill                   float64
matchDuration                 int64
maxPlace                      int64
numGroups                     int64
rankPoints                    int64
revives                       int64
rideDistance                  float64
roadKills                     int64
swimDistance                  float64
teamKills                     int64
vehicleDestroys               int64
walkDistance

In [53]:
data=data.drop(data[["Id", "groupId", "matchId", "matchDuration", "maxPlace", "rankPoints"]], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 38 columns):
assists                       int64
boosts                        int64
damageDealt                   float64
DBNOs                         int64
headshotKills                 int64
heals                         int64
killPlace                     int64
killPoints                    int64
kills                         int64
killStreaks                   int64
longestKill                   float64
numGroups                     int64
revives                       int64
rideDistance                  float64
roadKills                     int64
swimDistance                  float64
teamKills                     int64
vehicleDestroys               int64
walkDistance                  float64
weaponsAcquired               int64
winPoints                     int64
winPlacePerc                  float64
matchType_crashfpp            uint8
matchType_crashtpp            uint8
matchType_d

In [54]:
train, test= split(data, test_size=0.2, random_state=123)

In [55]:
train_y= train["winPlacePerc"].copy()
test_y= test["winPlacePerc"].copy()

In [56]:
train_x=train.drop(train[["winPlacePerc"]],axis=1)
test_x=test.drop(test[["winPlacePerc"]],axis=1)

In [57]:
#print("train_x: ", np.isnan(train_x).any())
print("train_y: ",np.isnan(train_y).any())
#print("test_x: ",np.isnan(test_x).any())
print("test_y: ",np.isnan(test_y).any())

train_y:  False
test_y:  True


In [58]:
test_y=test_y.fillna(test_y.mean())

In [59]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=22, random_state=0)  
regressor.fit(train_x, train_y)  
pred_y = regressor.predict(test_x) 

In [60]:
print('Mean Absolute Error:', metrics.mean_absolute_error(test_y, pred_y))  
print('Mean Squared Error:', metrics.mean_squared_error(test_y, pred_y))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(test_y, pred_y))) 

Mean Absolute Error: 0.06381260435852865
Mean Squared Error: 0.007984201972113157
Root Mean Squared Error: 0.08935436179679847


In [61]:
r2_score(pred_y,test_y)

0.9082710645740347

In [62]:
test_actual= pd.read_csv("test_V2.csv")

In [63]:
# One hot encode matchType
test_actual = pd.get_dummies(test_actual, columns=['matchType'])


In [65]:
test_model= test_actual.drop(test_actual[["Id", "groupId", "matchId", "matchDuration", "maxPlace", "rankPoints"]], axis=1)

In [66]:
test_model_predict= regressor.predict(test_model)

In [67]:
op= pd.DataFrame(list(zip(test_actual["Id"], test_model_predict)))
op= op.rename(columns={0: 'Id', 1: 'winPlacePerc'})

In [68]:
op.head()

,Id,winPlacePerc
0,9329eb41e215eb,0.276059
1,639bd0dcd7bda8,0.929691
2,63d5c8ef8dfe91,0.709891
3,cf5b81422591d1,0.560452
4,ee6a295187ba21,0.954023


In [69]:
op.to_csv("op_RandomForest_v2.csv", encoding='utf-8', index=False)